In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '7'

In [2]:
from deeprte.config import make_config
config = make_config()

In [3]:
config

best_model_eval_metric: ''
best_model_eval_metric_higher_is_better: true
checkpoint_interval_type: null
dataset:
  buffer_size: 5000
  data_split:
    is_split_datasets: false
    num_samples: 10
    num_test_samples: 2
    train_validation_split_rate: 0.8
  max_intra_op_parallelism: 1
  pre_shuffle: true
  seed: 42
  threadpool_size: 48
  train:
    batch_size: 6
    collocation_sizes: 500
    repeat: 1
  validation:
    batch_size: 2
eval_initial_weights: false
experiment_kwargs: {}
interval_type: secs
log_all_train_data: false
log_tensors_interval: 60
log_train_data_interval: 120.0
logging_interval_type: null
max_checkpoints_to_keep: 5
one_off_evaluate: false
random_mode_eval: same_host_same_device
random_mode_train: unique_host_unique_device
random_seed: 42
save_checkpoint_interval: 300
train_checkpoint_all_hosts: false
training_steps: 10000

In [4]:
data_path = "/workspaces/deeprte/rte_data/rte_data/matlab/eval-data/test_shape.mat"

from deeprte.data.pipeline import DataPipeline
data_pipeline = DataPipeline()
data = data_pipeline.process(data_path)

from deeprte.model.tf.rte_dataset import np_to_tensor_dict

tf_data = np_to_tensor_dict(data)

from deeprte.model.tf.rte_dataset import divide_batch_feat
batched_data, unbatched_data = divide_batch_feat(tf_data)

2022-11-25 22:15:47.128615: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-25 22:15:47.295619: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-25 22:15:49.649028: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriat

In [5]:
import tensorflow as tf

tf.nest.map_structure(lambda x:x.shape, tf_data)

{'sigma': TensorShape([10, 1681, 2]),
 'psi_label': TensorShape([10, 40344]),
 'boundary': TensorShape([10, 1968]),
 'position_coords': TensorShape([1681, 2]),
 'velocity_coords': TensorShape([24, 2]),
 'phase_coords': TensorShape([40344, 4]),
 'scattering_kernel': TensorShape([10, 40344, 24]),
 'boundary_coords': TensorShape([1968, 4]),
 'boundary_weights': TensorShape([1968]),
 'velocity_weights': TensorShape([24])}

In [6]:
data_config = config.dataset

In [11]:
from deeprte.model.tf.input_pipeline import load_and_split_data,process_features,make_device_batch
import jax

ds, unbatched_feat = load_and_split_data(tf_data, data_config, True)

In [12]:
data_config.data_split.is_split_datasets = False

In [17]:
ds_without_split, unbatched_feat = load_and_split_data(
        features=tf_data,
        config=data_config,
        pre_shuffle=True,
        seed=data_config.seed,
    )

In [18]:
tfds_without_split = process_features(
            ds=ds_without_split,
            unbatched_feat=unbatched_feat,
            is_training=False,
            batch_sizes=[2,5],
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
        )

In [19]:
tf.nest.map_structure(lambda x:x.shape, next(tfds_without_split))

{'sigma': (2, 5, 1681, 2),
 'psi_label': (2, 5, 40344),
 'boundary': (2, 5, 1968),
 'scattering_kernel': (2, 5, 40344, 24),
 'position_coords': (2, 1681, 2),
 'velocity_coords': (2, 24, 2),
 'phase_coords': (2, 40344, 4),
 'boundary_coords': (2, 1968, 4),
 'boundary_weights': (2, 1968),
 'velocity_weights': (2, 24)}

In [20]:
data_config.data_split.is_split_datasets = True
data_config

buffer_size: 5000
data_split:
  is_split_datasets: true
  num_samples: 10
  num_test_samples: 2
  train_validation_split_rate: 0.8
max_intra_op_parallelism: 1
pre_shuffle: true
seed: 42
threadpool_size: 48
train:
  batch_size: 6
  collocation_sizes: 500
  repeat: 1
validation:
  batch_size: 2

In [21]:
ds_with_split, unbatched_feat = load_and_split_data(
        features=tf_data,
        config=data_config,
        pre_shuffle=True,
        seed=data_config.seed,
    )

In [22]:
len(ds_with_split)

3

In [23]:
train_ds = process_features(
            ds=ds_with_split[1],
            unbatched_feat=unbatched_feat,
            is_training=True,
            batch_sizes=make_device_batch(data_config.train.batch_size, jax.local_device_count()),
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
            collocation_sizes=data_config.train.collocation_sizes,
            repeat=data_config.train.repeat,
        )

In [24]:
tf.nest.map_structure(lambda x:x.shape, next(train_ds))

{'sigma': (1, 6, 1681, 2),
 'psi_label': (1, 6, 500),
 'boundary': (1, 6, 1968),
 'scattering_kernel': (1, 6, 500, 24),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 500, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [25]:
val_ds = process_features(
            ds=ds_with_split[2],
            unbatched_feat=unbatched_feat,
            is_training=False,
            batch_sizes=make_device_batch(data_config.validation.batch_size, jax.local_device_count()),
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
            collocation_sizes=data_config.train.collocation_sizes,
            repeat=data_config.train.repeat,
        )

In [26]:
tf.nest.map_structure(lambda x:x.shape, next(val_ds))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'boundary': (1, 2, 1968),
 'scattering_kernel': (1, 2, 40344, 24),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}

In [27]:
test_ds = process_features(
            ds=ds_with_split[0],
            unbatched_feat=unbatched_feat,
            is_training=False,
            batch_sizes=make_device_batch(data_config.validation.batch_size, jax.local_device_count()),
            seed=data_config.seed,
            buffer_size=data_config.buffer_size,
            threadpool_size=data_config.threadpool_size,
            max_intra_op_parallelism=data_config.max_intra_op_parallelism,
            collocation_sizes=data_config.train.collocation_sizes,
            repeat=data_config.train.repeat,
        )

In [28]:
tf.nest.map_structure(lambda x:x.shape, next(test_ds))

{'sigma': (1, 2, 1681, 2),
 'psi_label': (1, 2, 40344),
 'boundary': (1, 2, 1968),
 'scattering_kernel': (1, 2, 40344, 24),
 'position_coords': (1, 1681, 2),
 'velocity_coords': (1, 24, 2),
 'phase_coords': (1, 40344, 4),
 'boundary_coords': (1, 1968, 4),
 'boundary_weights': (1, 1968),
 'velocity_weights': (1, 24)}